In [1]:
# importar pacotes necessários
import numpy as np
import pandas as pd

In [2]:
# definir parâmetros extras
pd.set_option('precision', 4)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [3]:
input_prefix = 'data/'
#input_prefix = 'https://github.com/hjort/ai-labs/raw/master/jupyter/sales-russia/data/'

bzfile = '.bz2' # ''

In [23]:
df_test = pd.read_csv(
    input_prefix + 'test.csv' + bzfile, #nrows=10000,
    #dtype={'shop_id': 'category', 'item_id': 'category'},
    dtype={'shop_id': np.int8, 'item_id': np.int16},
    index_col='ID')
print('shape:', df_test.shape)
df_test.head()

shape: (214200, 2)


,shop_id,item_id
ID,,
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268


In [24]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 214200 entries, 0 to 214199
Data columns (total 2 columns):
shop_id    214200 non-null int8
item_id    214200 non-null int16
dtypes: int16(1), int8(1)
memory usage: 2.2 MB


In [25]:
df_test.describe()

,shop_id,item_id
count,214200.0000,214200.0000
mean,31.6429,11019.3986
std,17.5619,6252.6446
min,2.0000,30.0000
25%,16.0000,5381.5000
50%,34.5000,11203.0000
75%,47.0000,16071.5000
max,59.0000,22167.0000


In [26]:
print('Total de linhas.....: %d' % df_test.shape[0])

shops_count = df_test['shop_id'].value_counts().count()
items_count = df_test['item_id'].value_counts().count()

print('Total de lojas......: %d' % shops_count)
print('Total de produtos...: %d' % items_count)
print('Total de combinações: %d (%d x %d)' % (shops_count * items_count, shops_count, items_count))

Total de linhas.....: 214200
Total de lojas......: 42
Total de produtos...: 5100
Total de combinações: 214200 (42 x 5100)


In [27]:
#dateparse = lambda x: pd.datetime.strptime(x, '%d.%m.%Y')
df_train = pd.read_csv(\
    input_prefix + 'sales_train.csv' + bzfile,
    #nrows=300, # somente primeiras linhas!
    #parse_dates=['date'], date_parser=dateparse,
    dtype={'date_block_num': np.int8, 'shop_id': np.int8, 'item_id': np.int16, \
           'item_price': np.float32, 'item_cnt_day': np.int32},
    )
print('shape:', df_train.shape)
df_train.head()

shape: (2935849, 6)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1


In [31]:
df_train2 = df_train.query('item_cnt_day > 0 and item_price > 0').drop(columns=['date'])
df_train2.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,0,59,22154,999.00,1
1,0,25,2552,899.00,1
3,0,25,2554,1709.05,1
4,0,25,2555,1099.00,1
5,0,25,2564,349.00,1


In [45]:
df_train3 = df_train2.groupby(['shop_id', 'item_id'], as_index=True).agg({
    'date_block_num': ['count', 'min', 'max'],
    'item_cnt_day': ['sum', 'min', 'max', 'mean', 'std', 'median'],
    'item_price': ['min', 'max', 'mean', 'std', 'median']
})

df_train3.head(20)

date_block_num         item_cnt_day                          \
                         count min max          sum min max    mean     std   
shop_id item_id                                                               
0       30                   9   1   1           31   2   9  3.4444  2.2423   
        31                   7   1   1           11   1   3  1.5714  0.9759   
        32                  11   0   1           16   1   2  1.4545  0.5222   
        33                   6   0   1            6   1   1  1.0000  0.0000   
        35                  12   0   1           15   1   2  1.2500  0.4523   
        36                   1   1   1            1   1   1  1.0000     NaN   
        40                   1   1   1            1   1   1  1.0000     NaN   
        42                   1   1   1            1   1   1  1.0000     NaN   
        43                   1   0   0            1   1   1  1.0000     NaN   
        49                   2   1   1            2   1   1  1.0000  0.0000   
        51                   4   0   1            5   1   2  1.2500  0.5000   
        57                   1   1   1            1   1   1  1.0000     NaN   
        59                   1   1   1            1   1   1  1.0000     NaN   
        61                   1   0   0            1   1   1  1.0000     NaN   
        75                   1   0   0            1   1   1  1.0000     NaN   
        85                   1   1   1            1   1   1  1.0000     NaN   
        88                   1   0   0            1   1   1  1.0000     NaN   
        95                   1   0   0            1   1   1  1.0000     NaN   
        96                   1   0   0            1   1   1  1.0000     NaN   
        97                   1   1   1            1   1   1  1.0000     NaN   

                       item_price                             
                median        min    max    mean  std median  
shop_id item_id                                               
0       30         3.0      265.0  265.0  265.00  0.0  265.0  
        31         1.0      434.0  434.0  434.00  0.0  434.0  
        32         1.0      221.0  221.0  221.00  0.0  221.0  
        33         1.0      347.0  347.0  347.00  0.0  347.0  
        35         1.0      247.0  247.0  247.00  0.0  247.0  
        36         1.0      357.0  357.0  357.00  NaN  357.0  
        40         1.0      127.0  127.0  127.00  NaN  127.0  
        42         1.0      127.0  127.0  127.00  NaN  127.0  
        43         1.0      221.0  221.0  221.00  NaN  221.0  
        49         1.0      127.0  127.0  127.00  0.0  127.0  
        51         1.0      127.0  130.0  127.75  1.5  127.0  
        57         1.0      167.0  167.0  167.00  NaN  167.0  
        59         1.0      110.0  110.0  110.00  NaN  110.0  
        61         1.0      195.0  195.0  195.00  NaN  195.0  
        75         1.0       76.0   76.0   76.00  NaN   76.0  
        85         1.0      190.0  190.0  190.00  NaN  190.0  
        88         1.0       76.0   76.0   76.00  NaN   76.0  
        95         1.0      193.0  193.0  193.00  NaN  193.0  
        96         1.0       70.0   70.0   70.00  NaN   70.0  
        97         1.0       67.0   67.0   67.00  NaN   67.0

In [46]:
df_train3.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 424098 entries, (0, 30) to (59, 22167)
Data columns (total 14 columns):
(date_block_num, count)    424098 non-null int64
(date_block_num, min)      424098 non-null int8
(date_block_num, max)      424098 non-null int8
(item_cnt_day, sum)        424098 non-null int32
(item_cnt_day, min)        424098 non-null int32
(item_cnt_day, max)        424098 non-null int32
(item_cnt_day, mean)       424098 non-null float64
(item_cnt_day, std)        293516 non-null float64
(item_cnt_day, median)     424098 non-null float64
(item_price, min)          424098 non-null float32
(item_price, max)          424098 non-null float32
(item_price, mean)         424098 non-null float32
(item_price, std)          293516 non-null float32
(item_price, median)       424098 non-null float32
dtypes: float32(5), float64(3), int32(3), int64(1), int8(2)
memory usage: 28.1 MB


In [47]:
df_train3.describe()

date_block_num                           item_cnt_day               \
               count          min          max          sum          min   
count    424098.0000  424098.0000  424098.0000  424098.0000  424098.0000   
mean          6.9052      10.5751      17.1074       8.6201       1.0094   
std          15.6708       9.8443      10.6592      64.5190       0.7904   
min           1.0000       0.0000       0.0000       1.0000       1.0000   
25%           1.0000       2.0000       8.0000       1.0000       1.0000   
50%           3.0000       8.0000      17.0000       3.0000       1.0000   
75%           7.0000      18.0000      27.0000       7.0000       1.0000   
max         867.0000      33.0000      33.0000   19934.0000     500.0000   

                                                            item_price  \
               max         mean          std       median          min   
count  424098.0000  424098.0000  293516.0000  424098.0000  424098.0000   
mean        1.4806       1.0686       0.1782       1.0250     725.0468   
std         5.5707       1.1451       1.1315       0.8667    1683.8182   
min         1.0000       1.0000       0.0000       1.0000       0.0700   
25%         1.0000       1.0000       0.0000       1.0000     196.5175   
50%         1.0000       1.0000       0.0000       1.0000     299.0000   
75%         1.0000       1.0000       0.0000       1.0000     699.0000   
max      2169.0000     666.6667     288.6751     500.0000  307980.0000   

                                                           
               max         mean          std       median  
count  424098.0000  424098.0000  293516.0000  424098.0000  
mean      862.2705     797.9776      85.6458     800.4720  
std      1880.7223    1769.4923     212.4699    1773.7587  
min         0.1000       0.0900       0.0000       0.1000  
25%       199.0000     199.0000       0.0000     199.0000  
50%       399.0000     349.0000       0.0333     349.0000  
75%       899.0000     799.0000      86.6025     799.0000  
max    307980.0000  307980.0000    7416.8428  307980.0000

In [48]:
# https://zaxrosenberg.com/pandas-multiindex-tutorial/

def flatten_cols(df: pd.DataFrame, delim: str = ""):
    """Flatten multiple column levels of the DataFrame into a one column level.

    Args:
        delim: the delimiter between the column values.

    Returns:
        A copy of the dataframe with the new column names.

    """
    new_cols = [delim.join((col_lev for col_lev in tup if col_lev))
                for tup in df.columns.values]
    ndf = df.copy()
    ndf.columns = new_cols

    return ndf

In [52]:
df_train4 = flatten_cols(df_train3, ":")
df_train4.head()

date_block_num:count  date_block_num:min  date_block_num:max  \
shop_id item_id                                                                 
0       30                          9                   1                   1   
        31                          7                   1                   1   
        32                         11                   0                   1   
        33                          6                   0                   1   
        35                         12                   0                   1   

                 item_cnt_day:sum  item_cnt_day:min  item_cnt_day:max  \
shop_id item_id                                                         
0       30                     31                 2                 9   
        31                     11                 1                 3   
        32                     16                 1                 2   
        33                      6                 1                 1   
        35                     15                 1                 2   

                 item_cnt_day:mean  item_cnt_day:std  item_cnt_day:median  \
shop_id item_id                                                             
0       30                  3.4444            2.2423                  3.0   
        31                  1.5714            0.9759                  1.0   
        32                  1.4545            0.5222                  1.0   
        33                  1.0000            0.0000                  1.0   
        35                  1.2500            0.4523                  1.0   

                 item_price:min  item_price:max  item_price:mean  \
shop_id item_id                                                    
0       30                265.0           265.0            265.0   
        31                434.0           434.0            434.0   
        32                221.0           221.0            221.0   
        33                347.0           347.0            347.0   
        35                247.0           247.0            247.0   

                 item_price:std  item_price:median  
shop_id item_id                                     
0       30                  0.0              265.0  
        31                  0.0              434.0  
        32                  0.0              221.0  
        33                  0.0              347.0  
        35                  0.0              247.0

In [58]:
df_train4[['date_block_num:count', 'item_cnt_day:sum']].head()

date_block_num:count  item_cnt_day:sum
shop_id item_id                                        
0       30                          9                31
        31                          7                11
        32                         11                16
        33                          6                 6
        35                         12                15